In [ ]:

! pip install pytorch-lightning


In [ ]:
!pip install torchmetrics

In [ ]:
!pip install "../input/hpacellsegmentatorraman/HPA-Cell-Segmentation/"


In [ ]:
!pip install pycocotools


In [ ]:
!pip install efficientnet_pytorch


In [ ]:
!pip install -U albumentations


In [ ]:
!pip install -q "../input/zoohpa/pytorch_zoo-master"

In [ ]:
#End to End Classification
import torch
import pytorch_lightning as pl
import torchvision.models as models

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
from torchvision import models

from skimage import io
from skimage.transform import resize
from skimage.color import rgb2gray,gray2rgb
from matplotlib import pyplot as plt

from PIL import Image

import csv
import pandas as pd
import numpy as np
import cv2

import albumentations as A
from albumentations.pytorch import ToTensorV2

import hpacellseg.cellsegmentator as cellsegmentator
from hpacellseg.utils import label_cell, label_nuclei
import base64
import numpy as np
from pycocotools import mask
from pycocotools import _mask as coco_mask
import typing as t
import zlib

In [ ]:

NUC_MODEL = "../input/requiredfiles/nuclei-model.pth"
CELL_MODEL = "../input/requiredfiles/cell-model.pth"


segmentator = cellsegmentator.CellSegmentator(
    NUC_MODEL,
    CELL_MODEL,
    device="cuda",
    multi_channel_model=True,
)


def extract_data_from_csv(csvpath):
    fields = [] 
    rows = [] 
    filename = csvpath
    with open(filename, 'r') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 

        fields = next(csvreader)
        # extracting each data row one by one 
        for row in csvreader: 
            rows.append(row) 
            
    return rows

# reading csv file

def get_segmented_image_and_label(container,index):
    
    '''
    Takes csv_path and index as an input
    Returns Segmented image and label if there is only single label
    '''
    
    #for row in rows[index]: # Read image and index
    Data = container[index]
    image_id = Data[0]
    labels = Data[1]
    labels = labels.replace("|",",")
    list_labels = list(labels.split(",")) 

    if(len(list_labels) == 1):

        img_blue = io.imread("../input/hpa-single-cell-image-classification/train/"+ image_id+"_blue.png")
        img_green = io.imread("../input/hpa-single-cell-image-classification/train/"+ image_id+"_green.png")
        img_red = io.imread("../input/hpa-single-cell-image-classification/train/"+ image_id+"_red.png")
        img_yellow = io.imread("../input/hpa-single-cell-image-classification/train/"+ image_id+"_yellow.png")
        
#         img_blue = Image.open("../input/hpa-single-cell-image-classification/train/"+ image_id+"_blue.png")
#         img_green = Image.open("../input/hpa-single-cell-image-classification/train/"+ image_id+"_green.png")
#         img_red = Image.open("../input/hpa-single-cell-image-classification/train/"+ image_id+"_red.png")
#         img_yellow = Image.open("../input/hpa-single-cell-image-classification/train/"+ image_id+"_yellow.png")
        
#         divider = 1
#         img_blue_resized = resize(img_blue, (img_blue.shape[0] // divider, img_blue.shape[1] // divider),
#                        anti_aliasing=True)
#         img_green_resized = resize(img_green, (img_green.shape[0] // divider, img_green.shape[1] // divider),
#                        anti_aliasing=True)
#         img_red_resized = resize(img_green, (img_red.shape[0] // divider, img_red.shape[1] // divider),
#                        anti_aliasing=True)
#         img_yellow_resized = resize(img_yellow, (img_yellow.shape[0] // divider, img_yellow.shape[1] // divider),
#                        anti_aliasing=True)

        #https://www.kaggle.com/thedrcat/hpa-single-cell-classification-eda/comments#Contents
        rgb = np.dstack((img_red_resized,img_yellow_resized+img_green_resized,img_blue_resized))


        nuc_segmentations = segmentator.pred_nuclei([rgb[:,:,2]])

        # For full cells
        cell_segmentations = segmentator.pred_cells([rgb],precombined = True)

        #print(cell_segmentations[0])
        #for i, pred in enumerate(cell_segmentations):
        nuclei_mask, cell_mask = label_cell(nuc_segmentations[0], cell_segmentations[0])

        return rgb,cell_mask,image_id,list_labels
    else:
        return -1,-1,-1,-1
   

            
def binary_mask_to_rle(binary_mask):
    rle = {'counts': [], 'size': list(binary_mask.shape)}
    counts = rle.get('counts')

    last_elem = 0
    running_length = 0

    for i, elem in enumerate(binary_mask.ravel(order='F')):
        if elem == last_elem:
            pass
        else:
            counts.append(running_length)
            running_length = 0
            last_elem = elem
        running_length += 1

    counts.append(running_length)
    
    return rle
      

def compress_rle(rle):
    compressed_rle = mask.frPyObjects(rle, rle.get('size')[0], rle.get('size')[1])
    return compressed_rle

def decode_rle(compressed_rle):
    mask_decode= mask.decode(compressed_rle)
    return mask_decode

def convert_to_binary_and_normalize(seg_mask):
    cell_mask = seg_mask[..., np.newaxis] # Added a dimension for channel
    _,thresh = cv2.threshold(cell_mask,0,255,cv2.THRESH_BINARY) # binarize
    thresh_norm = cv2.normalize(src=thresh, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    return thresh_norm

In [ ]:
# #Create a cropped CSV list
# #from IPython.display import FileLink
# filename = "Custom_Dataset.csv"
# fields = ['ImageId','Coordinate','Comp_RLE-Mask', 'label'] # Will Contain only single labelled cropped images
# container = extract_data_from_csv("../input/hpa-single-cell-image-classification/train.csv")


# with open(filename, 'w') as csvfile:
#     csvwriter = csv.writer(csvfile)  
#     csvwriter.writerow(fields)
#     #print("Started Writing")
#     for idx in range(8293,10000,1):#len(container)
#         rgb,seg_mask,imageid,label = get_segmented_image_and_label(container,idx)
#         print(idx)
#         if(label != -1):
#             thresh_norm = convert_to_binary_and_normalize(seg_mask)
#             contours, hierarchy = cv2.findContours(thresh_norm, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#             print("Total Contours: ", len(contours))
#             for c in contours:
#                 x, y, w, h = cv2.boundingRect(c)
                
#                 cropped_rgb = rgb[y:y+h,x:x+w]
#                 cropped_thresh = thresh_norm[y:y+h,x:x+w]
                
#                 cropped_rgb_resize = cv2.resize(cropped_rgb, (300, 300), interpolation = cv2.INTER_NEAREST)
#                 cropped_thresh_resize = cv2.resize(cropped_thresh, (300, 300), interpolation = cv2.INTER_NEAREST)
                
#                 rle = binary_mask_to_rle(cropped_thresh_resize)
#                 compressed_rle = compress_rle(rle)
#                 #print(compressed_rle.dtype)
#                 #print("Writing into CSV")
#                 csvwriter.writerow([imageid,[x,y,w,h],compressed_rle,label[0]])
                
# #                 fig, ax = plt.subplots(1,3, figsize=(20,50))
# #                 ax[0].imshow(rgb)
# #                 ax[0].axis('off')

# #                 ax[1].imshow(cropped_rgb_resize)
# #                 ax[1].axis('off')
                
# #                 mask_decoded = decode_rle(compressed_rle)
                
# #                 ax[2].imshow(mask_decoded)
# #                 ax[2].axis('off')
# #                 plt.show()

# print("END")

In [ ]:
# # Read CSV and decode RLE
# import ast
# custom_data = extract_data_from_csv("./Custom_Dataset.csv")

# for index in range(1):
#     Data = custom_data[index]
#     image_id = Data[0]
#     coord = ast.literal_eval(Data[1])
#     compressed_rle = ast.literal_eval(Data[2])
#     #print(coord)
#     fig, ax = plt.subplots(1,3, figsize=(20,50))
#     img_blue = io.imread("../input/hpa-single-cell-image-classification/train/"+ image_id+"_blue.png")
#     img_green = io.imread("../input/hpa-single-cell-image-classification/train/"+ image_id+"_green.png")
#     img_red = io.imread("../input/hpa-single-cell-image-classification/train/"+ image_id+"_red.png")
#     img_yellow = io.imread("../input/hpa-single-cell-image-classification/train/"+ image_id+"_yellow.png")
    
#     rgb = np.dstack((img_red,img_yellow+img_green,img_blue))
#     cropped_rgb = rgb[coord[1]:coord[1]+coord[3],coord[0]:coord[0]+coord[2]]
#     cropped_rgb_resize = cv2.resize(cropped_rgb, (300, 300), interpolation = cv2.INTER_NEAREST)
#     mask_decoded = decode_rle(compressed_rle)
#     ax[0].imshow(rgb)
#     ax[0].axis('off')
#     ax[1].imshow(cropped_rgb_resize)
#     ax[1].axis('off')
#     ax[2].imshow(mask_decoded)
#     ax[2].axis('off')
#     plt.show()

In [ ]:
# container = extract_data_from_csv("../input/hpa-single-cell-image-classification/train.csv")

# for i in range(6):
#     rgb,seg_mask,imageid,label = get_segmented_image_and_label(container,i)
       
#     if(label!= -1):
        
#         thresh_norm = convert_to_binary_and_normalize(seg_mask)
        
#         fig, ax = plt.subplots(1,3, figsize=(20,50))
#         ax[0].imshow(thresh_norm)
#         ax[0].axis('off')

#         ax[1].imshow(rgb)
#         ax[1].axis('off')
        
       
#         rle = binary_mask_to_rle(thresh_norm)
#         compressed_rle = compress_rle(rle)
#         mask_decoded = decode_rle(compressed_rle)
        
#         ax[2].imshow(mask_decode)
#         ax[2].axis('off')
        
#         plt.show()
        

In [ ]:
# from urllib.request import urlopen
# from io import BytesIO
# from zipfile import ZipFile
# from subprocess import Popen
# from os import chmod
# from os.path import isfile
# import json
# import time
# import psutil

# def launch_tensorboard():
#     tb_process, ngrok_process = None, None
    
#     # Launch TensorBoard
#     if not is_process_running('tensorboard'):
#         tb_command = 'tensorboard --logdir ./lightning_logs/ --host 0.0.0.0 --port 6006'
#         tb_process = run_cmd_async_unsafe(tb_command)
    
#     # Install ngrok
#     if not isfile('./ngrok'):
#         ngrok_url = 'https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip'
#         download_and_unzip(ngrok_url)
#         chmod('./ngrok', 0o755)

#     # Create ngrok tunnel and print its public URL
#     if not is_process_running('ngrok'):
#         ngrok_process = run_cmd_async_unsafe('./ngrok http 6006')
#         time.sleep(1) # Waiting for ngrok to start the tunnel
#     ngrok_api_res = urlopen('http://127.0.0.1:4040/api/tunnels', timeout=10)
#     ngrok_api_res = json.load(ngrok_api_res)
#     assert len(ngrok_api_res['tunnels']) > 0, 'ngrok tunnel not found'
#     tb_public_url = ngrok_api_res['tunnels'][0]['public_url']
#     print(f'TensorBoard URL: {tb_public_url}')

#     return tb_process, ngrok_process


# def download_and_unzip(url, extract_to='.'):
#     http_response = urlopen(url)
#     zipfile = ZipFile(BytesIO(http_response.read()))
#     zipfile.extractall(path=extract_to)


# def run_cmd_async_unsafe(cmd):
#     return Popen(cmd, shell=True)


# def is_process_running(process_name):
#     running_process_names = (proc.name() for proc in psutil.process_iter())
#     return process_name in running_process_names


# tb_process, ngrok_process = launch_tensorboard()

In [ ]:
# Create a DataLoader which returns contour based cropped images

import ast

#Training Loader
class CustomDataSet(Dataset): # To have 3 channel image ( r,y,b ) as input and g as mask with 19 class vector
    
    def __init__(self,root_dir_image,image_ids,labels,cord,transform):
        self.root_dir_image = root_dir_image
        self.augmentations = transform
        self.X_fold = image_ids
        self.y_fold = labels
        self.crop_fold = cord
        self.corrd = np.zeros((4,), dtype=int)
        
    
    def __len__(self):
        return len(self.X_fold)
    
   
    def __getitem__(self,index): 
        self.coord  = ast.literal_eval(self.crop_fold[index])
#         print(self.coord)
#         print(len(self.coord))
        
        img_blue = np.array(Image.open(self.root_dir_image + self.X_fold[index]+"_blue.png"))
        img_red = np.array(Image.open(self.root_dir_image + self.X_fold[index] +"_red.png"))
        img_yellow = np.array(Image.open(self.root_dir_image + self.X_fold[index] +"_yellow.png"))
        img_green = np.array(Image.open(self.root_dir_image + self.X_fold[index] +"_green.png"))
        
        #https://www.kaggle.com/thedrcat/hpa-single-cell-classification-eda/comments#Contents
       
        # Input Image
        rgb = np.dstack((img_red,img_yellow + img_green,img_blue))
        cropped_rgb = rgb[self.coord[1]:self.coord[1]+self.coord[3],self.coord[0]:self.coord[0]+self.coord[2]]
        

        torch_label = torch.from_numpy(np.asarray(self.y_fold[index]))
        
        if self.augmentations:
            cropped_rgb_ = np.array(cropped_rgb)
            torch_rgb_ = self.augmentations(image=cropped_rgb_)
            torch_rgb = torch_rgb_['image']

        
        return torch_rgb,torch.tensor(torch_label, dtype=torch.long)
            


In [ ]:
from sklearn.model_selection import StratifiedKFold

class HPADataModule(pl.LightningDataModule):
    def __init__(self,TrainFile,splits,validation_idx,BatchSize,ImagePath):
        super().__init__()
        self.csv_file = TrainFile
        self.splits = splits
        self.batch_size = BatchSize
        self.Imagepath = ImagePath
        self.validation_idx = validation_idx
        
        self.transform = A.Compose([
        A.Resize(224,224),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.HorizontalFlip(p=0.5),
        A.Normalize(
        
            mean=[0,0,0],
            std = [1,1,1],
            max_pixel_value = 255,
        
        ),
        ToTensorV2()])

        
    def prepare_data(self):
        #prepare_data is called only once on 1- GPU in a distributed computing
        df = pd.read_csv(self.csv_file)
        df["kfold"] =-1
        
        
        df = df.sample(frac=1).reset_index(drop=True)
        stratify = StratifiedKFold(n_splits=self.splits)
        for i,(t_idx,v_idx) in enumerate(stratify.split(X=df.ImageId.values,y=df.label.values)):
            df.loc[v_idx,"kfold"]=i
            
        
        df.to_csv("train_folds.csv",index=False)

    def setup(self,stage=None):
        dfx = pd.read_csv("train_folds.csv")
        train = dfx.loc[dfx["kfold"]!=self.validation_idx]
        val = dfx.loc[dfx["kfold"]==self.validation_idx]

        
        self.train_dataset = CustomDataSet(self.Imagepath,
                                            image_ids = train.ImageId.values,
                                            labels = train.label.values,
                                            cord = train.Coordinate.values,
                                            transform = self.transform)

        self.valid_dataset = CustomDataSet(self.Imagepath,
                                            image_ids = val.ImageId.values,
                                            labels = val.label.values,
                                            cord = val.Coordinate.values,
                                            transform = self.transform)

    def train_dataloader(self):
        return DataLoader(self.train_dataset,
                      batch_size= self.batch_size,
                      num_workers=4,
                      shuffle=True)
  
    def val_dataloader(self):
        return DataLoader(self.valid_dataset,
                      batch_size= self.batch_size,
                      num_workers=4)
  


In [ ]:
from torch.autograd import Variable

class WeightedFocalLoss(nn.Module):
    "Non weighted version of Focal Loss"
    def __init__(self,alpha=.25,gamma=2,eps=1e-7):
        super(WeightedFocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.eps = eps
    
    def one_hot(self,index, classes):
#         print(index)
#         print(classes)
        size = index.size() + (classes,)
        view = index.size() + (1,)

        mask = torch.Tensor(*size).fill_(0)
        index = index.view(*view)
        ones = 1.
        mask = mask.cuda()
        index = index.cuda()
        if isinstance(index, Variable):
            ones = Variable(torch.Tensor(index.size()).fill_(1)).cuda()
            mask = Variable(mask, volatile=index.volatile)

        #print("Inside Loss Function")
        #print(mask.shape)
        #print(index.shape)
        #print(ones.shape)
        return mask.scatter_(1, index, ones)
    
   

    def forward(self, inputs, target):
        y = self.one_hot(target, inputs.size(-1))
        #print(y)
        logit = F.softmax(inputs, dim=-1)
        logit = logit.clamp(self.eps, 1. - self.eps)

        loss = -1 * y * torch.log(logit.cuda()) # cross entropy
        loss = loss.cuda() * (1 - logit.cuda()) ** self.gamma # focal loss
        return loss.sum()
    
    

In [ ]:
# Class Architecture
from efficientnet_pytorch import EfficientNet
from pytorch_lightning.metrics import F1
from pytorch_lightning.metrics.functional import confusion_matrix
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve

class EffNet(pl.LightningModule):
    
    def __init__(self,learning_rate=1e-4):
        super().__init__()
        self.effnet = EfficientNet.from_pretrained('efficientnet-b3')
        in_features = self.effnet._fc.in_features
        self.effnet._fc = nn.Linear(in_features,19)
        self.train_acc = pl.metrics.Accuracy()
        self.valid_acc = pl.metrics.Accuracy()
        self.learning_rate = learning_rate
        self.classes = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18']

        
        
    def forward(self, x):
        raw_predit = self.effnet(x)
        return raw_predit

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=(self.learning_rate))
        #optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
        #scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.01, max_lr=0.1)
        
        #return [optimizer], [scheduler]
        return optimizer
    
    def loss_function(self,y_hat, y):
#         creterion = nn.CrossEntropyLoss()
#         loss = creterion(x_hat,x)

        loss  = WeightedFocalLoss()(y_hat, y)
        return loss
        
    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        y_hat = self(x.float())
        loss = self.loss_function(y_hat, y)
        
        self.log('train_loss', loss, on_step=False, on_epoch=True,prog_bar=True,logger=True)
        self.train_acc(y_hat, y)
        self.log('train_acc', self.train_acc, on_step=False, on_epoch=True,prog_bar=True,logger=True)
        return {'loss': loss}

    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
      
        y_hat = self(x.float()) 
        loss = self.loss_function(y_hat, y)
        self.log('val_loss', loss, on_step=False, on_epoch=True,prog_bar=True,logger=True)
        self.valid_acc(y_hat, y)
        self.log('val_acc', self.valid_acc,on_step=False, on_epoch=True,prog_bar=True,logger=True)
        return {'loss': loss, 'y': y, 'y_hat': y_hat.sigmoid()}
    
    def validation_epoch_end(self, outputs):
   
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        f1_score = self.get_f1(outputs)
        #print("Before")
        self.average_precision(outputs)
        #print("After")
        confusion_matrix = self.get_confusion_matrix(outputs)
        print(f"Epoch {self.current_epoch} | F1:{f1_score}")
        self.plot_confusion_matrix(confusion_matrix.numpy(),self.classes)
       
        return {'loss': avg_loss}
    
    def get_f1(self, outputs):
        
        y = torch.cat([x['y'] for x in outputs])
        y_hat = torch.cat([x['y_hat'] for x in outputs])
        f1 = F1(num_classes=19)

        return f1(y_hat.cpu().detach(), y.cpu().detach())
    
    def average_precision(self,outputs):
        print("Inside")
        y = torch.cat([x['y'] for x in outputs])
        
        y_hat = torch.cat([x['y_hat'] for x in outputs])
        
        y_hat_ = y_hat#torch.argmax(y_hat,dim=1)
        
        #target_data = y.cpu().detach().numpy()#pd.DataFrame(y.cpu().detach().numpy(),columns=['label'])
        #target_data['is_valid'] = True
        #target_data.reset_index(drop=True, inplace=True)
        
        predicted_data = y_hat_.cpu().detach().numpy()
        
        size = y.size() + (19,)
        view = y.size() + (1,)

        mask = torch.Tensor(*size).fill_(0)
        index = y.view(*view)
        ones = 1.
        mask = mask.cuda()
        index = index.cuda()
        if isinstance(index, Variable):
            ones = Variable(torch.Tensor(index.size()).fill_(1)).cuda()
            mask = Variable(mask, volatile=index.volatile)
        
        y = y.unsqueeze(1)
        #print(mask.shape)
        #print(y.shape)
        #print(ones.shape)
        encoded_vect = mask.scatter_(1, y.cuda(), ones)
    
        precision = dict()
        recall = dict()
        average_precision = dict()
        #print("Before LOOP")
       
        target_data_list = encoded_vect.cpu().detach().numpy()
        predicted_data_list = predicted_data
        #print(encoded_vect.shape)
        #print(predicted_data.shape)
        for i in range(16):
            precision[i], recall[i], _ = precision_recall_curve(target_data_list[i], predicted_data_list[i])
            average_precision[i] = average_precision_score(target_data_list[i], predicted_data_list[i])

        # A "micro-average": quantifying score on all classes jointly
        precision["micro"], recall["micro"], _ = precision_recall_curve(target_data_list.ravel(), predicted_data_list.ravel())
        average_precision["micro"] = average_precision_score(target_data_list, predicted_data_list, average="micro")
        #print("Before Final Print")
        print('Average precision score, micro-averaged over all classes: {0:0.2f}'.format(average_precision["micro"]))
        
        
        
    def get_confusion_matrix(self,outputs):
        y = torch.cat([x['y'] for x in outputs])
        y_hat = torch.cat([x['y_hat'] for x in outputs])
        
        return confusion_matrix(y_hat.cpu().detach(), y.cpu().detach(), num_classes=19)
    
    


    def plot_confusion_matrix(self,cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
        if normalize:
            cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            print("Normalized confusion matrix")
        else:
            print('Confusion matrix, without normalization')

        print(cm)
        plt.imshow(cm, interpolation='nearest', cmap=cmap)
        plt.title(title)
        plt.colorbar()
        tick_marks = np.arange(len(classes))
        plt.xticks(tick_marks, classes, rotation=45)
        plt.yticks(tick_marks, classes)

        fmt = '.2f'
        thresh = cm.max() / 2.
#         for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#             plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
        

        plt.figure(figsize=(12, 12))
        #plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')


In [ ]:
# model

from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
monitor='val_acc',
dirpath='./',
save_top_k=3,
mode='max',
)

splits = 5
batch_size = 16
nums_epoch = 3;
learning_rate = 3e-4
validation_index = 3

dataModule = HPADataModule("../input/equi-dis-all-label/equal_dis_all_label.csv",splits,validation_index,batch_size,"../input/hpa-single-cell-image-classification/train/")

model = EffNet(learning_rate)



In [ ]:
trainer = pl.Trainer(resume_from_checkpoint='../input/fresh-epoch-1/epoch1-step7451.ckpt',gpus = 1, max_epochs = nums_epoch,callbacks=[checkpoint_callback]) #resume_from_checkpoint='../input/training-experiment-hpa-cell-segmenter/epoch=7-step=34139.ckpt'
trainer.fit(model,dataModule)

In [ ]:
# import pandas as pd
# data_frame = pd.read_csv('../input/overall-data/Overall_customData.csv')
# print(data_frame.label.value_counts())

In [ ]:
# df_trimmed = data_frame[data_frame.label != 0]
# df_trimmed = df_trimmed[df_trimmed.label != 14]
# df_trimmed = df_trimmed[df_trimmed.label != 16]
# df_trimmed = df_trimmed[df_trimmed.label != 4]

In [ ]:
# print(df_trimmed.label.value_counts())

In [ ]:
# df_trimmed.to_csv('../input/overall-data/equal_dis.csv', index=False)